<a href="https://colab.research.google.com/github/thisisRMak/2025-tech16-LLM/blob/main/Lectures/Class2_TECH16_LLM_Lecture2_prepared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)


# Prompt engineering - few shot

In [ ]:
def chat(message):
    """
    Send a message to the OpenAI GPT-3.5 model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0.3,
        # response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """Below are examples of text messages and their classifications. After studying these examples, please classify the new text message at the end.

              Example 1:

              Text: "Can you send me the files by tomorrow? It's not urgent, but I'd like to review them."
              Classification: Non-Urgent
              Example 2:

              Text: "Please review the final report ASAP! We need it ready by our meeting in the morning!"
              Classification: Urgent
              Example 3:

              Text: "Let's schedule a meeting for next week to discuss the project. No rush."
              Classification: Non-Urgent
              Example 4:

              Text: "URGENT: The server is down and needs immediate attention!"
              Classification: Urgent
              """},
            {"role": "user", "content": f"Classify the following message: {message}"}
        ]
    )

    text_only = response.choices[0].message.content
    return text_only


In [ ]:
chat("Reminder: Tomorrow's team meeting has been postponed. Please update your calendars")

'Classification: Non-Urgent'

### JSON mode and Log Probs

In [ ]:
def chat(message):
    """
    Send a message to the OpenAI GPT-3.5 model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT-3.5-turbo model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT-3.5 model.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        logprobs=True,
        messages=[
            {"role": "system", "content": """
            You are a helpful assistant classifations.

            Below are examples of text messages and their classifications. After studying these examples, please classify the new text message at the end.

              Example 1:

              Text: "Can you send me the files by tomorrow? It's not urgent, but I'd like to review them."
              Classification: Non-Urgent
              Example 2:

              Text: "Please review the final report ASAP! We need it ready by our meeting in the morning!"
              Classification: Urgent
              Example 3:

              Text: "Let's schedule a meeting for next week to discuss the project. No rush."
              Classification: Non-Urgent
              Example 4:

              Text: "URGENT: The server is down and needs immediate attention!"
              Classification: Urgent

              """},
            {"role": "user", "content": f"Classify the following message as Non-Urgent or Urgent and return with probability as JSON: {message}"}
        ]
    )
    text_only = response.choices[0].message.content
    return text_only


In [ ]:
response = chat("Reminder: Tomorrow's team meeting has been postponed. Please update your calendars")

In [ ]:
print(response)

{
  "Classification": "Non-Urgent",
  "Probability": 0.95
}


# Chain of thought reasoning

In [ ]:
def chat(message):
    """
    Send a message to the OpenAI GPT model and return its response.

    This function interacts with the OpenAI API, specifically using the GPT model. It takes a user's message as input, sends it to the model, and returns the model's text-only response. The function ensures the AI's output is concise by providing a system-level instruction.

    Parameters:
    message (str): A string containing the user's message to the AI.

    Returns:
    str: The text response generated by the GPT model.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": """
            You are a helpful assistant classifations.

            Below are examples of questions and how to calculate the answer

              Example 1: Arithmetic Problem
              Prompt: "If a shirt costs $20 and there is a 10% discount, how much does the shirt cost after the discount?"
              Chain of Thought Answer:
                Calculate the amount of discount: 10% of $20 is $2.
                Subtract the discount from the original price: $20 - $2 = $18.
                The shirt costs $18 after the discount.

              Example 2: Logic Puzzle
              Prompt: "There are four apples and you take away three. How many apples do you have?"
              Chain of Thought Answer:
                You start with four apples.
                You take away three apples.
                After taking three, you now have those three apples.
                You have 3 apples

              """},
            {"role": "user", "content": f"Answer the following question: {message}"}
        ]
    )
    text_only = response.choices[0].message.content
    return text_only


In [ ]:
print(chat("If a tree absorbs 48 pounds of CO2 a year, how much CO2 will 10 trees absorb in a year?"))

To find out how much CO2 10 trees will absorb in a year, you multiply the amount one tree absorbs by the number of trees.

- Each tree absorbs 48 pounds of CO2 a year.
- Number of trees: 10.

Calculation: 48 pounds/tree × 10 trees = 480 pounds.

The 10 trees will absorb 480 pounds of CO2 in a year.


# Langchain & summarizing PDF's

In [ ]:
# Get a PDF
!wget https://arxiv.org/pdf/2401.16212.pdf

--2025-02-06 23:49:52--  https://arxiv.org/pdf/2401.16212.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2401.16212 [following]
--2025-02-06 23:49:53--  http://arxiv.org/pdf/2401.16212
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512852 (501K) [application/pdf]
Saving to: ‘2401.16212.pdf’

2401.16212.pdf      100%[===================>] 500.83K  1.66MB/s    in 0.3s    

2025-02-06 23:49:53 (1.66 MB/s) - ‘2401.16212.pdf’ saved [512852/512852]



In [ ]:
!ls

2401.16212.pdf	sample_data


In [ ]:
!wget https://www.morganstanley.com/im/publication/insights/articles/article_increasingreturns.pdf

--2025-02-06 23:49:54--  https://www.morganstanley.com/im/publication/insights/articles/article_increasingreturns.pdf
Resolving www.morganstanley.com (www.morganstanley.com)... 23.220.224.225
Connecting to www.morganstanley.com (www.morganstanley.com)|23.220.224.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘article_increasingreturns.pdf’

article_increasingr     [ <=>                ] 564.52K  --.-KB/s    in 0.09s   

2025-02-06 23:49:54 (5.96 MB/s) - ‘article_increasingreturns.pdf’ saved [578072]



In [ ]:
!pip install -U langchain-community pypdf langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33


In [ ]:
from langchain.document_loaders import PyPDFLoader # Update import statement

loader = PyPDFLoader("2401.16212.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(metadata={'source': '2401.16212.pdf', 'page': 0, 'page_label': '1'}, page_content='Better Call GPT, Comparing Large Language Models Against Lawyers\nLAUREN MARTIN, NICK WHITEHOUSE, STEPHANIE YIU, LIZZIE CATTERSON, RIVINDU\nPERERA, AI Center of Excellence, Onit Inc., New Zealand\nThis paper presents a groundbreaking comparison between Large Language Models (LLMs) and traditional legal contract review-\ners—Junior Lawyers and Legal Process Outsourcers (LPOs). We dissect whether LLMs can outperform humans in accuracy, speed,\nand cost-efficiency during contract review. Our empirical analysis benchmarks LLMs against a ground truth set by Senior Lawyers,\nuncovering that advanced models match or exceed human accuracy in determining legal issues. In speed, LLMs complete reviews in\nmere seconds, eclipsing the hours required by their human counterparts. Cost-wise, LLMs operate at a fraction of the price, offering a\nstaggering 99.97 percent reduction in cost over traditional methods.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4-turbo-preview", api_key=open_ai_key)
chain = load_summarize_chain(llm, chain_type="stuff")

res = chain.invoke(pages[0:3])
res

{'input_documents': [Document(metadata={'source': '2401.16212.pdf', 'page': 0, 'page_label': '1'}, page_content='Better Call GPT, Comparing Large Language Models Against Lawyers\nLAUREN MARTIN, NICK WHITEHOUSE, STEPHANIE YIU, LIZZIE CATTERSON, RIVINDU\nPERERA, AI Center of Excellence, Onit Inc., New Zealand\nThis paper presents a groundbreaking comparison between Large Language Models (LLMs) and traditional legal contract review-\ners—Junior Lawyers and Legal Process Outsourcers (LPOs). We dissect whether LLMs can outperform humans in accuracy, speed,\nand cost-efficiency during contract review. Our empirical analysis benchmarks LLMs against a ground truth set by Senior Lawyers,\nuncovering that advanced models match or exceed human accuracy in determining legal issues. In speed, LLMs complete reviews in\nmere seconds, eclipsing the hours required by their human counterparts. Cost-wise, LLMs operate at a fraction of the price, offering a\nstaggering 99.97 percent reduction in cost over

In [ ]:
print(res["output_text"])

This paper, titled "Better Call GPT, Comparing Large Language Models Against Lawyers" by Lauren Martin et al., from Onit Inc., New Zealand, explores the effectiveness of Large Language Models (LLMs) in legal contract review tasks, comparing their performance with Junior Lawyers and Legal Process Outsourcers (LPOs). The study focuses on accuracy, speed, and cost-efficiency, finding that LLMs can match or exceed human accuracy in identifying legal issues, complete reviews significantly faster, and operate at a substantially lower cost, offering a 99.97% reduction compared to traditional methods. The research indicates a potential shift in legal practice towards LLM dominance, suggesting a future where legal workflows are significantly transformed by AI technology. The study contributes to the understanding of LLM capabilities in the legal domain, highlighting their potential to enhance the accessibility and efficiency of legal services.


In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary in a maximum of 3 bullets of the following text enclosed within three backticks:
```{text}```
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo-preview", api_key=open_ai_key)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

res = stuff_chain.invoke(pages[0:3])

In [ ]:
print(res["output_text"])

- The paper evaluates the performance of Large Language Models (LLMs) against Junior Lawyers and Legal Process Outsourcers (LPOs) in legal contract review, focusing on accuracy, speed, and cost-efficiency, with LLMs showing superior results in all areas.
- Empirical analysis reveals LLMs match or exceed human accuracy in identifying legal issues, complete reviews significantly faster, and operate at a 99.97% lower cost than traditional methods.
- The findings suggest LLMs have the potential to disrupt the legal industry by enhancing the accessibility and efficiency of legal services, indicating a shift towards LLM dominance in legal contract review.
